In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/neha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

 Import Packages

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

Importing Lyrics data

In [4]:
# Import Dataset
df = pd.read_csv('/Users/neha/Downloads/lyrics_small.csv')
df.head()

,song,year,artist,genre,lyrics
0,the-downward-spiral,2007,discharge,Rock,Mother Dear Mother\nIt's Cold Tonight\nLike Ev...
1,lulu-s-back-in-town,2008,fats-waller,Jazz,"Gotta get my old tuxedo pressed, gotta sew a b..."
2,donde-quieras-yo-ire,2006,backstreet-boys,Pop,Al fin del mundo llegare\nLo que quieras you h...
3,falling-in-love,2007,don-williams,Country,"Falling In Love Again\nLove be gentle, love be..."
4,i-hate-u-i-love-you,2016,gnash,Rock,Feeling used\nBut I'm\nStill missing you\nAnd ...


In [6]:
print(df.genre.unique())

['Rock' 'Jazz' 'Pop' 'Country' 'Not Available' 'Metal' 'Electronic'
 'Hip-Hop' 'Other' 'Indie' 'R&B' 'Folk']


In [7]:
print(df.artist.unique())

['discharge' 'fats-waller' 'backstreet-boys' ... 'doppelt-so-schlecht'
 'adverts' 'elissa']


In [8]:
print(df.year.unique())

[2007 2008 2006 2016 2004 2013 2012 1994 2011 2010 2014 2009 2015 1983
 2005 2002 2003 1998 1991 1977 1972 2000 1995 1987 1992 2001 1970 1981
 1997 1996 1993 1975 1990 1982 1974 1999 1988 1978 1973 1989 1986 1980
 1971 1984 1979 1985 1976]


 Remove newline characters

In [9]:
# Convert to list
data = df.lyrics.values.tolist()

data = [re.sub('[^a-zA-Z ]' ,'', str(sent)) for sent in data]

pprint(data[:1])

['Mother Dear MotherIts Cold TonightLike Every OtherEvery Other NightBut I '
 'Never FeelFeel It AnywayIm Gonna Need SomethingSomething SoonI Can Feel '
 'ItFeel It BeginBut I Dont KnowHow Im Gonna PayIt Must Be AboutMid December '
 'Right NowAnd I Think Im AboutSeventeenIm Not Real SureHow Old I FeelI Lost '
 'My ThoughtsIn Some DreamOh Mother I DontKnow HowI Got Where I AmBut Ill Try '
 'To Explain AnyhowSee GraduallyYou Get Sucked InInto ItWithout Ever '
 'ReallyRealisingJust Whats HappeningAnd That Is WhenThe Downward Spiral '
 'BeginsAnyway Back To How It All StartedIt Started With DopeWhy Not After All '
 'It Was Just The Once I Told MyselfI Didnt Even Like It Very MuchBut The '
 'People I Was With All Did ItThen I Tried SpeedWhy Not After All It Was Just '
 'The Once I Told MyselfThe Next Thing I Knew I Was Doing A Couple Of Grams A '
 'WeekThen A Friend Introduced Me To Smack Chasing The DragonWhy Not After All '
 'It Was Just The Once I Told MyselfWhy Not After All It Wasnt Us

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['mother', 'dear', 'motherits', 'cold', 'tonightlike', 'every', 'otherevery', 'other', 'nightbut', 'never', 'feelfeel', 'it', 'anywayim', 'gonna', 'need', 'sooni', 'can', 'feel', 'itfeel', 'it', 'beginbut', 'dont', 'knowhow', 'im', 'gonna', 'payit', 'must', 'be', 'aboutmid', 'december', 'right', 'nowand', 'think', 'im', 'not', 'real', 'surehow', 'old', 'feeli', 'lost', 'my', 'thoughtsin', 'some', 'dreamoh', 'mother', 'dontknow', 'howi', 'got', 'where', 'ambut', 'ill', 'try', 'to', 'explain', 'anyhowsee', 'graduallyyou', 'get', 'sucked', 'ininto', 'itwithout', 'ever', 'whats', 'happeningand', 'that', 'is', 'whenthe', 'downward', 'spiral', 'beginsanyway', 'back', 'to', 'how', 'it', 'all', 'startedit', 'started', 'with', 'dopewhy', 'not', 'after', 'all', 'it', 'was', 'just', 'the', 'once', 'told', 'myselfi', 'didnt', 'even', 'like', 'it', 'very', 'muchbut', 'the', 'people', 'was', 'with', 'all', 'did', 'itthen', 'tried', 'speedwhy', 'not', 'after', 'all', 'it', 'was', 'just', 'the', 'onc

### Creating Bigram and Trigram Models
Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(bigram_mod[data_words[0]])

['mother', 'dear', 'motherits', 'cold', 'tonightlike', 'every', 'otherevery', 'other', 'nightbut', 'never', 'feelfeel', 'it', 'anywayim', 'gonna', 'need', 'sooni', 'can', 'feel', 'itfeel', 'it', 'beginbut', 'dont', 'knowhow', 'im', 'gonna', 'payit', 'must', 'be', 'aboutmid', 'december', 'right', 'nowand', 'think', 'im', 'not', 'real', 'surehow', 'old', 'feeli', 'lost', 'my', 'thoughtsin', 'some', 'dreamoh', 'mother', 'dontknow', 'howi', 'got', 'where', 'ambut', 'ill', 'try', 'to', 'explain', 'anyhowsee', 'graduallyyou', 'get', 'sucked', 'ininto', 'itwithout', 'ever', 'whats', 'happeningand', 'that', 'is', 'whenthe', 'downward', 'spiral', 'beginsanyway', 'back', 'to', 'how', 'it', 'all', 'startedit', 'started', 'with', 'dopewhy', 'not', 'after', 'all', 'it', 'was', 'just', 'the', 'once', 'told', 'myselfi', 'didnt', 'even', 'like', 'it', 'very', 'muchbut', 'the', 'people', 'was', 'with', 'all', 'did', 'itthen', 'tried', 'speedwhy', 'not', 'after', 'all', 'it', 'was', 'just', 'the', 'once

### Remove Stopwords, Make Bigrams and Lemmatize
The bigrams model is ready. Let’s define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let’s call the functions in order.

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['mother', 'dear', 'motherit', 'cold', 'tonightlike', 'otherevery', 'nightbut', 'never', 'feelfeel', 'anywayim', 'go', 'need', 'sooni', 'feel', 'itfeel', 'beginbut', 'do', 'not', 'knowhow', 'be', 'go', 'payit', 'must', 'aboutmid', 'december', 'right', 'nowand', 'think', 'be', 'real', 'surehow', 'old', 'feeli', 'lose', 'thoughtsin', 'dreamoh', 'mother', 'dontknow', 'howi', 'get', 'ambut', 'ill', 'try', 'explain', 'anyhowsee', 'get', 'suck', 'itwithout', 'ever', 'what', 's', 'happeningand', 'whenthe', 'downward', 'spiral', 'beginsanyway', 'back', 'startedit', 'start', 'dopewhy', 'tell', 'myselfi', 'do', 'not', 'even', 'muchbut', 'people', 'itthen', 'try', 'speedwhy', 'tell', 'myselfthe', 'next', 'thing', 'know', 'couple', 'gram', 'friend', 'introduce', 'smack', 'chase', 'dragonwhy', 'tell', 'myselfwhy', 'be', 'not', 'use', 'needlethen', 'start', 'say', 'would', 'never']]


### Create the Dictionary and Corpus needed for Topic Modeling
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 3), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 3), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1)]]


Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [18]:
id2word[10]

'couple'

In [19]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aboutmid', 1),
  ('ambut', 1),
  ('anyhowsee', 1),
  ('anywayim', 1),
  ('back', 1),
  ('be', 3),
  ('beginbut', 1),
  ('beginsanyway', 1),
  ('chase', 1),
  ('cold', 1),
  ('couple', 1),
  ('dear', 1),
  ('december', 1),
  ('do', 2),
  ('dontknow', 1),
  ('dopewhy', 1),
  ('downward', 1),
  ('dragonwhy', 1),
  ('dreamoh', 1),
  ('even', 1),
  ('ever', 1),
  ('explain', 1),
  ('feel', 1),
  ('feelfeel', 1),
  ('feeli', 1),
  ('friend', 1),
  ('get', 2),
  ('go', 2),
  ('gram', 1),
  ('happeningand', 1),
  ('howi', 1),
  ('ill', 1),
  ('introduce', 1),
  ('itfeel', 1),
  ('itthen', 1),
  ('itwithout', 1),
  ('know', 1),
  ('knowhow', 1),
  ('lose', 1),
  ('mother', 2),
  ('motherit', 1),
  ('muchbut', 1),
  ('must', 1),
  ('myselfi', 1),
  ('myselfthe', 1),
  ('myselfwhy', 1),
  ('need', 1),
  ('needlethen', 1),
  ('never', 2),
  ('next', 1),
  ('nightbut', 1),
  ('not', 3),
  ('nowand', 1),
  ('old', 1),
  ('otherevery', 1),
  ('payit', 1),
  ('people', 1),
  ('real', 1),
  ('right

### Building the Topic Model
We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.070*"que" + 0.063*"la" + 0.049*"shake" + 0.028*"sin" + 0.026*"por" + '
  '0.025*"land" + 0.021*"amor" + 0.020*"con" + 0.015*"una" + 0.012*"ser"'),
 (1,
  '0.017*"job" + 0.013*"master" + 0.011*"youwe" + 0.005*"georgia" + '
  '0.002*"exit" + 0.002*"up" + 0.001*"daysand" + 0.000*"party" + 0.000*"rag" + '
  '0.000*"dich"'),
 (2,
  '0.044*"who" + 0.019*"meoh" + 0.017*"tie" + 0.007*"rainand" + 0.006*"prince" '
  '+ 0.005*"seeand" + 0.003*"toll" + 0.002*"timewell" + 0.000*"deserts_miss" + '
  '0.000*"christma"'),
 (3,
  '0.016*"hereand" + 0.009*"embrace" + 0.008*"liar" + 0.001*"knowni" + '
  '0.001*"altar" + 0.000*"cruel" + 0.000*"dead" + 0.000*"found_mefound" + '
  '0.000*"anybody" + 0.000*"headi"'),
 (4,
  '0.055*"not" + 0.047*"be" + 0.042*"get" + 0.028*"go" + 0.027*"do" + '
  '0.026*"know" + 0.016*"make" + 0.015*"come" + 0.015*"see" + 0.014*"want"'),
 (5,
  '0.014*"queen" + 0.014*"swing" + 0.004*"alla" + 0.002*"againyoure" + '
  '0.000*"lady" + 0.000*"che" + 0.000*"non" + 0.000*"

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -18.719782509023876

Coherence Score:  0.46586449077042946


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/neha/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.540033  0.051615       1        1  72.404167
10    -0.359646  0.085459       2        1  12.617105
19    -0.113719 -0.389415       3        1   5.841998
6      0.010985  0.022232       4        1   2.523860
0      0.044596  0.024003       5        1   2.159739
7      0.054777  0.021654       6        1   0.950082
11     0.059313  0.019263       7        1   0.837080
16     0.060697  0.020192       8        1   0.815463
12     0.064866  0.015109       9        1   0.577000
14     0.064390  0.015744      10        1   0.302845
18     0.065204  0.013789      11        1   0.212256
2      0.065319  0.013265      12        1   0.187281
1      0.065442  0.011407      13        1   0.101427
9      0.065406  0.010868      14        1   0.087460
17     0.065413  0.010952      15        1   0.086564
5      0.065392  0.010701      16        1   0.074813
8      0.065404  0.010822      17        1   0.065913
15     0.065391  0.010698      18        1   0.058748
13     0.065411  0.010907      19        1   0.049077
3      0.065394  0.010734      20        1   0.047126, topic_info=       Category          Freq              Term          Total  loglift  \
term                                                                      
191     Default   40552.00000              love   40552.000000   30.000   
51      Default  110809.00000               not  110809.000000   29.000   
716     Default   11460.00000              baby   11460.000000   28.000   
5       Default   94558.00000                be   94558.000000   27.000   
26      Default   84667.00000               get   84667.000000   26.000   
186     Default   12635.00000             heart   12635.000000   25.000   
181     Default   11554.00000              fall   11554.000000   24.000   
1234    Default    4758.00000               die    4758.000000   23.000   
1574    Default   12054.00000             world   12054.000000   22.000   
27      Default   56655.00000                go   56655.000000   21.000   
13      Default   54315.00000                do   54315.000000   20.000   
385     Default   12416.00000               day   12416.000000   19.000   
36      Default   52577.00000              know   52577.000000   18.000   
158     Default    4196.00000               que    4196.000000   17.000   
2085    Default    5549.00000              spin    5549.000000   16.000   
478     Default   13202.00000              life   13202.000000   15.000   
474     Default    3994.00000              head    3994.000000   14.000   
261     Default    8856.00000             night    8856.000000   13.000   
1001    Default    3937.00000                la    3937.000000   12.000   
1425    Default    6713.00000             dream    6713.000000   11.000   
390     Default    7494.00000               eye    7494.000000   10.000   
439     Default    6248.00000             light    6248.000000    9.000   
3093    Default    2956.00000             shake    2956.000000    8.000   
342     Default   33138.00000              make   33138.000000    7.000   
208     Default    4505.00000              burn    4505.000000    6.000   
296     Default   28991.00000              want   28991.000000    5.000   
3515    Default    3181.00000              meet    3181.000000    4.000   
1375    Default    2430.00000             fight    2430.000000    3.000   
59      Default   27956.00000                 s   27956.000000    2.000   
48      Default   27949.00000             never   27949.000000    1.000   
...         ...           ...               ...            ...      ...   
399510  Topic20       0.00317      handsmemorie       0.646662    2.342   
399516  Topic20       0.00317    restlessnessis       0.646662    2.342   
399515  Topic20       0.00317           preyliv       0.646662    2.342   
398131  Topic20       0.00317         airsearch       0.646663    2.3

In [24]:
mallet_path = '/Users/neha/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [25]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(2,
  [('light', 0.032365740948449906),
   ('dream', 0.030660864103369375),
   ('eye', 0.023823410650993762),
   ('sun', 0.018735699223832385),
   ('fly', 0.01502983534478891),
   ('star', 0.012526358293328549),
   ('sky', 0.0122212750684194),
   ('wind', 0.012095652564045045),
   ('rain', 0.01144959397011979),
   ('high', 0.010399748754991252)]),
 (16,
  [('watch', 0.02895738203957382),
   ('hope', 0.026407914764079148),
   ('hate', 0.0156519533231862),
   ('beautiful', 0.01297564687975647),
   ('write', 0.012480974124809741),
   ('side', 0.012417554540842212),
   ('perfect', 0.009411466260781329),
   ('room', 0.007382039573820396),
   ('win', 0.0069634703196347035),
   ('mother', 0.006291222729578894)]),
 (7,
  [('love', 0.22786009065692442),
   ('baby', 0.05639841688654353),
   ('give', 0.04740037886475881),
   ('kiss', 0.015459035247953454),
   ('sweet', 0.01144205398822813),
   ('youand', 0.01062174413097896),
   ('heart', 0.010528719301806372),
   ('hold', 0.008702049928962858),

In [26]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)